In [2]:
import csv
from collections import defaultdict
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
import pandas as pd

In [3]:
Fams,species={},[]
for r in csv.reader(open('Orthogroups_JSD.tsv'),delimiter='\t'):
    if r[0]=="Orthogroup":
        species=r[1:]
        print(species)
        continue
    fid,glt=r[0],r[1:]
    #print(fid,'>>>',glt)
    gbs=defaultdict(list)
    gbs={species[i]:gs.split(',') for i,gs in enumerate(glt)}
    Fams[fid]=gbs
    #break
        #if i==5: break
#species=list(set(species))


['Aque', 'Blan', 'Cele', 'Cgig', 'Chem', 'Ctel', 'Dgyr', 'Dmel', 'Drer', 'Eand', 'Hmia', 'Hrob', 'Hsap', 'Lana', 'Lgig', 'Lluy', 'Locu', 'Myes', 'Ngen', 'Nvec', 'Ofus', 'Paus', 'Skow', 'Smar', 'Smed', 'Spur', 'Tcas']


In [4]:
print(gbs)

{'Aque': [''], 'Blan': [''], 'Cele': [''], 'Cgig': [''], 'Chem': [''], 'Ctel': [''], 'Dgyr': [''], 'Dmel': [''], 'Drer': [''], 'Eand': [''], 'Hmia': [''], 'Hrob': [''], 'Hsap': [''], 'Lana': [''], 'Lgig': [''], 'Lluy': [''], 'Locu': [''], 'Myes': [''], 'Ngen': [''], 'Nvec': [''], 'Ofus': [''], 'Paus': [''], 'Skow': [''], 'Smar': [''], 'Smed': [''], 'Spur': [''], 'Tcas': ['Tcas_TC034497_001', ' Tcas_TC035044_001']}


In [5]:
genefam=defaultdict(dict)
for fid,gbs in Fams.items():
    for sp in gbs:
        for g in gbs[sp]:
            #genefam[sp][g.split('.',1)[0]]=fid
            genefam[sp][g]=fid
    

In [6]:
print(genefam.keys())

dict_keys(['Aque', 'Blan', 'Cele', 'Cgig', 'Chem', 'Ctel', 'Dgyr', 'Dmel', 'Drer', 'Eand', 'Hmia', 'Hrob', 'Hsap', 'Lana', 'Lgig', 'Lluy', 'Locu', 'Myes', 'Ngen', 'Nvec', 'Ofus', 'Paus', 'Skow', 'Smar', 'Smed', 'Spur', 'Tcas'])


In [7]:
print(list(genefam.items())[18:23])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
def gbm(mods):
    gbms=defaultdict(list)
    for g,m in mods.items():
        gbms[m].append(g)
    return(gbms)

In [9]:
ctel_mods={r[0]:r[1] for r in csv.reader(open('03-Capitella_teleta_mfuzz_clusters_TFonly.txt'),delimiter='\t')}

In [10]:
ofus_mods={r[0]:r[1] for r in csv.reader(open('03-Owenia_fusiformis_mfuzz_clusters_TFonly.txt'),delimiter='\t')}

In [11]:
ctel_gbm=gbm(ctel_mods)
ofus_gbm=gbm(ofus_mods)


In [12]:
def comp(gl1,gl2,pairs):
    p1={r[0]:r[1] for r in pairs}
    p2={r[1]:r[0] for r in pairs}
    rgl1=[g1 for g1 in gl1 if g1 in p1]
    rgl2=[g2 for g2 in gl2 if g2 in p2]


In [13]:
print(sum([1 for f,gbs in Fams.items() if 'Ctel' in gbs and 'Ofus' in gbs]))


54440


In [14]:
print(list(ctel_gbm.items())[0:4])

[('2', ['CTELG00000000036.1', 'CTELG00000000229.1', 'CTELG00000000271.1', 'CTELG00000000343.1', 'CTELG00000000343.2', 'CTELG00000000445.1', 'CTELG00000000600.2', 'CTELG00000000785.1', 'CTELG00000001238.1', 'CTELG00000001274.1', 'CTELG00000001560.1', 'CTELG00000001874.1', 'CTELG00000001874.2', 'CTELG00000001946.1', 'CTELG00000002156.1', 'CTELG00000002286.1', 'CTELG00000002657.1', 'CTELG00000002657.2', 'CTELG00000003084.1', 'CTELG00000003085.1', 'CTELG00000003329.2', 'CTELG00000003343.1', 'CTELG00000003557.1', 'CTELG00000003621.3', 'CTELG00000003821.1', 'CTELG00000003821.2', 'CTELG00000003866.1', 'CTELG00000004199.1', 'CTELG00000004221.1', 'CTELG00000004342.1', 'CTELG00000004451.1', 'CTELG00000004627.1', 'CTELG00000004678.1', 'CTELG00000004755.1', 'CTELG00000004830.1', 'CTELG00000005324.1', 'CTELG00000005387.1', 'CTELG00000005423.1', 'CTELG00000005488.1', 'CTELG00000005631.1', 'CTELG00000005914.1', 'CTELG00000006018.1', 'CTELG00000006044.1', 'CTELG00000006282.1', 'CTELG00000006742.1', 'C

In [15]:
def ogcomp(gbm1,gbm2,genefam,nm1,nm2):
    res=[]
    nshog=sum([1 for f,gbs in Fams.items() if nm1 in gbs and nm2 in gbs])
    for m1,gl1 in gbm1.items():
        for m2,gl2 in gbm2.items():
            #print('pliv{} <-> spur{}'.format(m1,m2))
            rgl1=set([genefam[nm1][g1] for g1 in gl1 if g1 in genefam[nm1]])
            rgl2=set([genefam[nm2][g2] for g2 in gl2 if g2 in genefam[nm2]])
            gle=rgl1.union(rgl2)
            gli=rgl1.intersection(rgl2)
            # M : 
            hpd=stats.hypergeom(nshog, len(rgl1),len(rgl2))
            print(nshog,len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)))
            rep=round(len(gli)/(len(rgl2)*len(rgl1)/nshog),3)
            # (n * D) / N
            #print(len(p1),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep)
            res.append(['{}{}'.format(nm1,m1),'{}{}'.format(nm2,m2),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep])
            phyper=pd.DataFrame(res, columns = ['cl1','cl2','no1','no2','nov','pv','rf'])  
    return(phyper)

In [16]:
def comp(gbm1,gbm2,pairs,nm1,nm2):
    res=[]
    for m1,gl1 in gbm1.items():
        for m2,gl2 in gbm2.items():
            #print('pliv{} <-> spur{}'.format(m1,m2))
            p1={r[0]:r[1] for r in pairs}
            p2={r[1]:r[0] for r in pairs}
            rgl1=[g1 for g1 in gl1 if g1 in p1]
            rgl2=[g2 for g2 in gl2 if g2 in p2]
            gle=set([p2[g2] for g2 in gl2 if g2 in p2]+rgl1)
            gli=[g1 for g1 in rgl1 for g2 in rgl2 if p1[g1]==g2]
            hpd=stats.hypergeom(len(p1), len(rgl1),len(rgl2))
            rep=round(len(gli)/(len(rgl2)*len(rgl1)/len(p1)),3)# (n * D) / N
            #print(len(p1),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep)
            res.append(['{}{}'.format(nm1,m1),'{}{}'.format(nm2,m2),len(rgl1),len(rgl2),len(gli),hpd.pmf(len(gli)),rep])
    phyper=pd.DataFrame(res, columns = ['cl1','cl2','no1','no2','nov','pv','rf'])  
    return(phyper)

In [17]:
ctel_ofus_hyp=ogcomp(ctel_gbm,ofus_gbm,genefam,'Ctel','Ofus')

54440 66 75 10 2.597923035000267e-18
54440 66 62 8 9.585708149745685e-15
54440 66 46 2 0.0014225968713520631
54440 66 15 1 0.017883487876014275
54440 66 17 3 1.1386082158595623e-06
54440 66 12 1 0.014358171066759438
54440 66 26 3 4.3083516721244174e-06
54440 66 20 3 1.9022217108773232e-06
54440 66 7 0 0.9915439316849107
54440 66 41 5 1.6144908570040662e-09
54440 66 2 1 0.0024217926598549882
54440 66 29 6 1.16413609310412e-12
54440 10 75 0 0.9863073353463927
54440 10 62 1 0.01127433859139586
54440 10 46 1 0.008387010740368676
54440 10 15 0 0.9972478596305467
54440 10 17 0 0.9968814231020843
54440 10 12 0 0.9977977417251681
54440 10 26 0 0.9952339578879468
54440 10 20 0 0.9963319955222288
54440 10 7 0 0.9987148183184287
54440 10 41 1 0.007481566227413783
54440 10 2 0 0.9996326534391357
54440 10 29 1 0.00530235562666864
54440 25 75 4 4.087982261584728e-08
54440 25 62 3 3.158691614234124e-06
54440 25 46 2 0.00020567662247659497
54440 25 15 1 0.006845918971295251
54440 25 17 0 0.99222071571

In [18]:
print(ctel_ofus_hyp)

       cl1     cl2  no1  no2  nov            pv       rf
0    Ctel2   Ofus2   66   75   10  2.597923e-18  109.980
1    Ctel2   Ofus1   66   62    8  9.585708e-15  106.432
2    Ctel2   Ofus4   66   46    2  1.422597e-03   35.863
3    Ctel2  Ofus11   66   15    1  1.788349e-02   54.990
4    Ctel2   Ofus3   66   17    3  1.138608e-06  145.561
..     ...     ...  ...  ...  ...           ...      ...
139  Ctel3   Ofus6   16   20    0  9.941373e-01    0.000
140  Ctel3   Ofus7   16    7    0  9.979444e-01    0.000
141  Ctel3  Ofus10   16   41    1  1.191782e-02   82.988
142  Ctel3   Ofus8   16    2    0  9.994123e-01    0.000
143  Ctel3   Ofus5   16   29    0  9.915097e-01    0.000

[144 rows x 7 columns]


In [19]:
ctel_ofus_hyp.to_csv('ctel_ofus_phyper_TF.tsv',sep='\t')